# Lending Club Case Study
## by Ankit Kumar Surana

## Introduction
As a worker of a consumer finance company that specialises in lending different kinds of loans to urban clients, part of my job is to facilitate loan approval decision-making by evaluating application profiles and identifying hazards related to loan payback potential. To do this, I would need to analyze data in "loan.csv", which contains historical information about past loan applicants with default status information. This means finding patterns that indicate the applicant is likely to default, which in turn enables taking further action, such as denying a loan, adjusting the loan amount, or applying higher interest rates to risky applicants.

Through the analysis, I aim to understand consumer and loan attributes affecting the customer's tendency to default, and also to find the driving factors, or variables, behind loan defaults. The company can then use such knowledge to improve its portfolio and risk assessment strategies.

## Preliminary Wrangling

In [1]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import regex as re

%matplotlib inline

# Gathering

In [2]:
df = pd.read_csv('loan.csv')

/var/folders/0p/s2hsmj5j32l2tpky_c8c91q00000gn/T/ipykernel_12379/129683778.py:1: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('loan.csv')


In [3]:
# high-level overview of data shape and composition
print(df.shape)

(39717, 111)


In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(74), int64(13), object(24)
memory usage: 33.6+ MB
None


In [5]:
print(df.head())

        id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  1077501    1296599       5000         5000           4975.0   36 months   
1  1077430    1314167       2500         2500           2500.0   60 months   
2  1077175    1313524       2400         2400           2400.0   36 months   
3  1076863    1277178      10000        10000          10000.0   36 months   
4  1075358    1311748       3000         3000           3000.0   60 months   

  int_rate  installment grade sub_grade  ... num_tl_90g_dpd_24m  \
0   10.65%       162.87     B        B2  ...                NaN   
1   15.27%        59.83     C        C4  ...                NaN   
2   15.96%        84.33     C        C5  ...                NaN   
3   13.49%       339.31     C        C1  ...                NaN   
4   12.69%        67.79     B        B5  ...                NaN   

  num_tl_op_past_12m pct_tl_nvr_dlq  percent_bc_gt_75 pub_rec_bankruptcies  \
0                NaN            NaN               

In [6]:
# Data Dictionary
data_dictionary = pd.read_excel('Data_Dictionary.xlsx')

In [7]:
data_dictionary [ data_dictionary['LoanStatNew'] == 'loan_amnt' ]

,LoanStatNew,Description
41,loan_amnt,The listed amount of the loan applied for by t...


# Assessing

In [8]:
# Check duplicated value
df.duplicated().sum()

0

In [9]:
# Check null value for each column
null_cols = df.columns[df.isnull().all(axis=0)].tolist()

print(f"List of columns with NULL's : \n\n {null_cols} \n")
print(f"Count of columns having all NULL values : {len(null_cols)}")

List of columns with NULL's : 

 ['mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_

In [10]:
# Find the uniqueness of a column in data frame

# Function that lists the columns with same values in the entire dataset 
def uniqueness(column_list): 
    uniq_cols = []
    for column in column_list:
     if len(df[column].unique()) == 1 and df[column].notnull().all(axis=0):
         # print(column, "->", df[column].unique()[0])
         uniq_cols.append(column)
    return uniq_cols

print("\nList of columns that have same value for all records : ", uniqueness(df.columns.to_list()) )


List of columns that have same value for all records :  ['pymnt_plan', 'initial_list_status', 'policy_code', 'application_type', 'acc_now_delinq', 'delinq_amnt']


In [11]:
# Find columns that have Categorical variables in the dataset

# Function that lists the categorical_values in a column
def categorical_values(column_list):
    for column in column_list:
         print(f"<<<<< {column} >>>>> \n")
         print(df[column].value_counts(), "\n")

column_list = ['term', 'grade', 'sub_grade', 'verification_status', 'loan_status', 'purpose', "home_ownership"]
categorical_values(column_list)

<<<<< term >>>>> 

term
 36 months    29096
 60 months    10621
Name: count, dtype: int64 

<<<<< grade >>>>> 

grade
B    12020
A    10085
C     8098
D     5307
E     2842
F     1049
G      316
Name: count, dtype: int64 

<<<<< sub_grade >>>>> 

sub_grade
B3    2917
A4    2886
A5    2742
B5    2704
B4    2512
C1    2136
B2    2057
C2    2011
B1    1830
A3    1810
C3    1529
A2    1508
D2    1348
C4    1236
C5    1186
D3    1173
A1    1139
D4     981
D1     931
D5     874
E1     763
E2     656
E3     553
E4     454
E5     416
F1     329
F2     249
F3     185
F4     168
F5     118
G1     104
G2      78
G4      56
G3      48
G5      30
Name: count, dtype: int64 

<<<<< verification_status >>>>> 

verification_status
Not Verified       16921
Verified           12809
Source Verified     9987
Name: count, dtype: int64 

<<<<< loan_status >>>>> 

loan_status
Fully Paid     32950
Charged Off     5627
Current         1140
Name: count, dtype: int64 

<<<<< purpose >>>>> 

purpose
debt_consolida

In [12]:
# Columns with NULL/ NAN values
cols_null = df.columns[df.isna().sum()>0 & df.notnull().all()].tolist()
print(cols_null)

['emp_title', 'emp_length', 'desc', 'title', 'mths_since_last_delinq', 'mths_since_last_record', 'revol_util', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl

# Cleaning

In [13]:
df_clean = df.copy()

In [14]:
df_clean.shape

(39717, 111)

In [15]:
df_clean.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


##### Define

>1) Dropping the 54 columns having all NULL values
>2) Dropping the columns that have same values in all rows of the dataset
>3) Removing the % symbol from the "int_rate & revol_util" column and change the data type to float.
>4) Removing the string "months" from the "term" column
>5) Dropping additional columns that do not aid in analysis : 'url', 'title', 'desc', 'zip_code', 'emp_title'
>6) Converting the data type of the date columns to date
>7) Dropping columns that have only "0.0" and "nan" as values.

##### Code

In [16]:
# Dropping all the columns having NULL values
df1 = df_clean.dropna(axis = 1, how = 'all')

In [17]:
# Dropping all the columns that have same values in all the rows of the dataset.
df1 = df1.drop(uniqueness(df1.columns.tolist()), axis=1)

In [18]:
# Removing the % symbol from the int_rate column and changing the data type to float.

def perc_rem(col_list):
    for col in col_list:
        df1[col] = df1[col].str.split("%").str[0]
        df1[col] = df1[col].astype("float")
        
col_list = ['int_rate', 'revol_util']
perc_rem(col_list)

In [19]:
# Removing the string "months" from the "term" column and changing the column name to "term_months"
df1.term = df1.term.str.split("months").str[0]

In [20]:
# Dropping any additional columns that do not aid in analysis.
col_drop = ['url', 'title', 'desc', 'zip_code', 'emp_title']
df1 = df1.drop(col_drop, axis=1)

In [21]:
# Converting to date
for col in df1.columns.to_list():
    if re.match('(.*_d$|.*cr_line$)', col):
        print(col)
        df1[col] = pd.to_datetime(df1[col],format="%b-%y")

issue_d
earliest_cr_line
last_pymnt_d
next_pymnt_d
last_credit_pull_d


In [22]:
# Dropping columns that have only "0.0" and "nan" as values.
col_drop = ['collections_12_mths_ex_med','chargeoff_within_12_mths','tax_liens']
df1 = df1.drop(col_drop, axis=1)

##### Test

In [23]:
# Validating if there is any column that has all NULL values
df1.columns[df1.isnull().all(axis=0)].tolist()

[]

In [24]:
# Validating if there are columns that have same values for all rows in the dataset.
uniqueness(df1.columns.to_list()) 

[]

In [25]:
# Validating the int_rate column.
df1.int_rate.describe()

count    39717.000000
mean        12.021177
std          3.724825
min          5.420000
25%          9.250000
50%         11.860000
75%         14.590000
max         24.590000
Name: int_rate, dtype: float64

In [26]:
# Validating the term column
df1.term.value_counts()

term
 36     29096
 60     10621
Name: count, dtype: int64

In [27]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 43 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   id                       39717 non-null  int64         
 1   member_id                39717 non-null  int64         
 2   loan_amnt                39717 non-null  int64         
 3   funded_amnt              39717 non-null  int64         
 4   funded_amnt_inv          39717 non-null  float64       
 5   term                     39717 non-null  object        
 6   int_rate                 39717 non-null  float64       
 7   installment              39717 non-null  float64       
 8   grade                    39717 non-null  object        
 9   sub_grade                39717 non-null  object        
 10  emp_length               38642 non-null  object        
 11  home_ownership           39717 non-null  object        
 12  annual_inc               39717 n

In [28]:
df1.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,pub_rec_bankruptcies
0,1077501,1296599,5000,5000,4975.0,36,10.65,162.87,B,B2,...,5000.00,863.16,0.00,0.00,0.00,2015-01-01,171.62,NaT,2016-05-01,0.0
1,1077430,1314167,2500,2500,2500.0,60,15.27,59.83,C,C4,...,456.46,435.17,0.00,117.08,1.11,2013-04-01,119.66,NaT,2013-09-01,0.0
2,1077175,1313524,2400,2400,2400.0,36,15.96,84.33,C,C5,...,2400.00,605.67,0.00,0.00,0.00,2014-06-01,649.91,NaT,2016-05-01,0.0
3,1076863,1277178,10000,10000,10000.0,36,13.49,339.31,C,C1,...,10000.00,2214.92,16.97,0.00,0.00,2015-01-01,357.48,NaT,2016-04-01,0.0
4,1075358,1311748,3000,3000,3000.0,60,12.69,67.79,B,B5,...,2475.94,1037.39,0.00,0.00,0.00,2016-05-01,67.79,2016-06-01,2016-05-01,0.0


In [29]:
df1.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,issue_d,dti,...,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,pub_rec_bankruptcies
count,3.971700e+04,3.971700e+04,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,3.971700e+04,39717,39717.000000,...,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39646,39717.000000,1140,39715,39020.000000
mean,6.831319e+05,8.504636e+05,11219.443815,10947.713196,10397.448868,12.021177,324.561922,6.896893e+04,2010-11-03 23:26:45.166553344,13.315130,...,9793.348813,2263.663172,1.363015,95.221624,12.406112,2013-04-10 05:32:22.652474368,2678.826162,2016-06-01 09:28:25.263157760,2014-09-07 19:16:08.007050496,0.043260
min,5.473400e+04,7.069900e+04,500.000000,500.000000,0.000000,5.420000,15.690000,4.000000e+03,2007-06-01 00:00:00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2008-01-01 00:00:00,0.000000,2016-06-01 00:00:00,2007-05-01 00:00:00,0.000000
25%,5.162210e+05,6.667800e+05,5500.000000,5400.000000,5000.000000,9.250000,167.020000,4.040400e+04,2010-05-01 00:00:00,8.170000,...,4600.000000,662.180000,0.000000,0.000000,0.000000,2012-04-01 00:00:00,218.680000,2016-06-01 00:00:00,2013-06-01 00:00:00,0.000000
50%,6.656650e+05,8.508120e+05,10000.000000,9600.000000,8975.000000,11.860000,280.220000,5.900000e+04,2011-02-01 00:00:00,13.400000,...,8000.000000,1348.910000,0.000000,0.000000,0.000000,2013-04-01 00:00:00,546.140000,2016-06-01 00:00:00,2015-03-01 00:00:00,0.000000
75%,8.377550e+05,1.047339e+06,15000.000000,15000.000000,14400.000000,14.590000,430.780000,8.230000e+04,2011-08-01 00:00:00,18.600000,...,13653.260000,2833.400000,0.000000,0.000000,0.000000,2014-06-01 00:00:00,3293.160000,2016-06-01 00:00:00,2016-05-01 00:00:00,0.000000
max,1.077501e+06,1.314167e+06,35000.000000,35000.000000,35000.000000,24.590000,1305.190000,6.000000e+06,2011-12-01 00:00:00,29.990000,...,35000.020000,23563.680000,180.200000,29623.350000,7002.190000,2016-05-01 00:00:00,36115.200000,2016-07-01 00:00:00,2016-05-01 00:00:00,2.000000
std,2.106941e+05,2.656783e+05,7456.670694,7187.238670,7128.450439,3.724825,208.874874,6.379377e+04,NaN,6.678594,...,7065.522127,2608.111964,7.289979,688.744771,148.671593,NaN,4447.136012,NaN,NaN,0.204324


In [33]:
# cols_null = df1.columns[df1.isna().sum()>0 & df1.notnull().all()].tolist()

# for cols in cols_null:
#     print("\n", df1[cols].describe())


 count         38642
unique           11
top       10+ years
freq           8879
Name: emp_length, dtype: object

 count    14035.000000
mean        35.900962
std         22.020060
min          0.000000
25%         18.000000
50%         34.000000
75%         52.000000
max        120.000000
Name: mths_since_last_delinq, dtype: float64

 count    2786.000000
mean       69.698134
std        43.822529
min         0.000000
25%        22.000000
50%        90.000000
75%       104.000000
max       129.000000
Name: mths_since_last_record, dtype: float64

 count    39667.000000
mean        48.832152
std         28.332634
min          0.000000
25%         25.400000
50%         49.300000
75%         72.400000
max         99.900000
Name: revol_util, dtype: float64

 count                            39646
mean     2013-04-10 05:32:22.652474368
min                2008-01-01 00:00:00
25%                2012-04-01 00:00:00
50%                2013-04-01 00:00:00
75%                2014-06-01 00:00:00
m

# Univariate Exploration

In [31]:
# Creating bins for loan amount
print(df.loan_amnt.describe())

bins = [0,5000,10000,15000,20000,25000,30000,35000]
pd.cut(df['loan_amnt'], bins=bins).value_counts()

count    39717.000000
mean     11219.443815
std       7456.670694
min        500.000000
25%       5500.000000
50%      10000.000000
75%      15000.000000
max      35000.000000
Name: loan_amnt, dtype: float64


loan_amnt
(5000, 10000]     12960
(0, 5000]          9568
(10000, 15000]     7986
(15000, 20000]     4591
(20000, 25000]     2999
(30000, 35000]      828
(25000, 30000]      785
Name: count, dtype: int64

# Segemented Exploration

# Bivariate Exploration